In [1]:
!pip install -U sec-edgar-downloader
!pip install xlsxwriter

Requirement already up-to-date: sec-edgar-downloader in /usr/local/lib/python3.6/dist-packages (3.0.1)


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/My\ Drive/AB/sec-gov/sec-edgar-financials/
!pwd
!ls

Mounted at /content/drive
/content/drive/My Drive/AB/sec-gov/sec-edgar-financials
/content/drive/My Drive/AB/sec-gov/sec-edgar-financials
crawlData.ipynb  edgar	  README.md  sec.gov	   setup.py
crawl.py	 LICENSE  results    sec-gov.xlsx  tests


In [3]:
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())
worksheet = gc.open('TICS').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
data = pd.DataFrame.from_records(rows, index = None)
cmpList = data[data.columns[0]].astype(str).values.tolist()
print('Number of tickets', len(cmpList))

Number of tickets 100


In [0]:
# cmpList = ['CRWD', 'AAL', 'AAPL']

In [0]:
import json
import numpy as np
import pandas as pd
from edgar.financials import FinancialReportEncoder
from edgar.stock import Stock

In [0]:
def getData(ticket, ty, ignored_cmp, period='annual', year=2018, quarter=0):
    # period = 'annual' # or 'quarterly', which is the default
    # year = 2018 # can use default of 0 to get the latest
    # quarter = 1 # 1, 2, 3, 4, or default value of 0 to get the latest
    try:
        stock = Stock(ticket)
        filing = stock.get_filing(period, year, quarter)
        # financial reports (contain data for multiple years)
        if ty == 'income_statements':
            statements = filing.get_income_statements()
        elif ty == "balance_sheets":
            statements = filing.get_balance_sheets()
        elif ty == "cash_flows":
            statements = filing.get_cash_flows()

        jsonstr = FinancialReportEncoder().encode(statements)
        data = json.loads(jsonstr)

        # print(data.keys())  # dict_keys(['company', 'date_filed', 'reports'])
        listreports = data['reports']
        columns = []
        data = []

        map_keys = []

        for report in listreports:
            # print(report.keys())  # dict_keys(['date', 'months', 'map']): string, int, dict

            for report_key in report.keys():
                if report_key == 'map':
                    for map_key in report['map'].keys():
                        map_keys.append(map_key)
                        # print(map_key, report['map'][map_key].keys())  # dict_keys(['label', 'value'])
                        for key in report['map'][map_key].keys():
                            # print(key, report['map'][map_key][key])
                            if key == 'label':
                                columns.append(report['map'][map_key][key])
                            else:
                                data.append(report['map'][map_key][key])

        sec_data = pd.DataFrame([data], columns=columns)
        sec_data = sec_data.transpose()
        sec_data.reset_index(inplace=True)
        sec_data['ind'] = np.arange(len(sec_data))
        sec_data.set_index("ind", inplace=True)
        map_data = pd.Series(map_keys)
        return sec_data, True, ignored_cmp

    except:
        print('Invalid company input', cmp, ty, period, year, quarter)
        ignored_cmp.append(cmp)
        return None, False, ignored_cmp

In [0]:

final_df = pd.DataFrame()
ignored_cmp = []

print('Total companies', len(cmpList))

types = ['income_statements', 'balance_sheets', 'cash_flows']
years = [2019, 2018, 2017, 2016]

filename = 'sec-gov.xlsx'
writer = pd.ExcelWriter(filename, engine='xlsxwriter')

for cmp in cmpList:
    columns = []
    cmp_df = pd.DataFrame()
    cmp_df['ind'] = np.arange(len(cmp_df))
    cmp_df.set_index("ind", inplace=True)
    for ty in types:
        for yr in years:
            print('Company: ', cmp)
            typedf, valid, ignored_cmp = getData(cmp, ty, ignored_cmp, year = yr)
            if valid:
                cmp_df = pd.concat([cmp_df,typedf], axis = 1)
                columns.append(str(ty) + '_' + str(yr))
                columns.append(str(ty) + '_' + str(yr))
    
    cmp_df.columns = columns
    # Add ticket name at front
    tic = pd.Series(data = [cmp])
    cmp_df.insert(0, 'tic', tic)
    cmp_df.to_excel(writer, sheet_name=cmp, index = False)

writer.save()
writer.close()    
!cp $filename /content/drive/My\ Drive/AB/

if len(ignored_cmp) > 0:
    ignored_csv = pd.DataFrame(ignored_cmp)
    ignored_filename = 'sec_ignoredcmp.xlsx'
    ignored_csv.to_excel(ignored_filename, index="False")
    !cp $ignored_filename drive/My\ Drive/AB/
    print('IGNORED COMPANIES: ', len(ignored_cmp))

Total companies 100
Company:  A
cik for A is 1090872
getting ['10-K', '10-K/A'] filing info from https://www.sec.gov/Archives/edgar/full-index/2019/QTR4/master.idx
Processing SGML at https://www.sec.gov/Archives/edgar/data/1090872/0001090872-19-000022.txt
could not find anything for ShortName consolidated statements of income
could not find anything for ShortName consolidated statements of operations
could not find anything for ShortName consolidated statement of earnings
could not find anything for ShortName condensed consolidated statements of income (unaudited)
could not find anything for ShortName condensed consolidated statements of income
could not find anything for ShortName condensed consolidated statements of operations (unaudited)
could not find anything for ShortName condensed consolidated statements of operations
could not find anything for ShortName condensed consolidated statement of earnings (unaudited)
could not find anything for ShortName condensed consolidated stateme